In [2]:
# Kriter 1: Veri Yükleme ve Kayıp Veri Tedavisi
import pandas as pd
import numpy as np
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error 
import pickle
import os

# DOSYA BİLGİLERİ (İndirdiğiniz dosyanın adı ve hedef sütun)
FILE_NAME = 'HousingData.csv' 
TARGET_COLUMN = 'MEDV' 

try:
    data = pd.read_csv(FILE_NAME) 
    print("Veri Seti Başarıyla Yüklendi."),

    # Sütun adlarını temizleme
    data.columns = data.columns.str.replace(' ', '_')
    
except FileNotFoundError:
    print(f"HATA: '{FILE_NAME}' dosyası bulunamadı. Lütfen dosyayı aynı klasöre kopyalayın.")
    raise

# --- Kayıp Veri Tedavisi ---
# Kayıp veriler (%4) ortalama (mean) ile doldurulur (Gerekçe: Veri kaybını önlemek)
for col in data.columns:
    if data[col].isnull().any():
        data[col].fillna(data[col].mean(), inplace=True)
        
print("Kayıp veriler başarıyla dolduruldu.")

# Hedef ve Bağımsız Değişkenleri Ayırma
y = data[TARGET_COLUMN] # Tahmin edilecek ev fiyatı (MEDV)
X = data.drop(TARGET_COLUMN, axis=1) # Bağımsız değişkenler (13 adet)

# Kriter 1'i tamamlamak için kategorik veri kontrolü (Boston'da gerek yoktur)
print(f"Başlangıç Öznitelik Sayısı: {X.shape[1]}")

Veri Seti Başarıyla Yüklendi.
Kayıp veriler başarıyla dolduruldu.
Başlangıç Öznitelik Sayısı: 13


C:\Users\huaweı\AppData\Local\Temp\ipykernel_20528\4260125839.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mean(), inplace=True)


In [3]:
# Kriter 2'ye hazırlık: Regresyon sabitini ekleme (statsmodels için zorunlu)
# X ve y'nin önceki hücrede tanımlı olduğunu varsayıyoruz.
import statsmodels.api as sm
X = sm.add_constant(X)
print("Regresyon Sabiti (Constant) Eklendi.")

# --- Kriter 2: Geriye Doğru Eleme (Backward Elimination) ---

X_opt = X.copy()
alpha = 0.05 # Anlamlılık düzeyi

print("\n--- Geriye Doğru Eleme Başlatıldı ---")

# Döngü, max 10 öznitelik sınırına ulaşana kadar devam eder.
while X_opt.shape[1] > 10:
    # 1. Modeli Kurma (En küçük kareler yöntemi - OLS)
    model_ols = sm.OLS(y, X_opt).fit()
    
    # 2. En yüksek p-değerini bulma
    p_values = model_ols.pvalues
    p_values_no_const = p_values.drop('const', errors='ignore')
    
    if p_values_no_const.empty: 
        break
        
    max_p_value = p_values_no_const.max()
    max_p_col = p_values_no_const.idxmax()
    
    # 3. Kontrol ve Eleme
    if max_p_value > alpha:
        # P-değeri 0.05'ten büyükse (anlamsızsa), özniteliği ele
        print(f"Eliyor (p > 0.05): {max_p_col} (p-değeri: {max_p_value:.4f})")
        X_opt = X_opt.drop(columns=[max_p_col])
    else:
        # P-değeri anlamlı ama öznitelik sayısı hala > 10 ise zorunlu eleme (En yüksek p-değerliyi ele)
        if X_opt.shape[1] > 10:
             print(f"Zorunlu Eleme (Max 10 Kriteri): {max_p_col} (p-değeri: {max_p_value:.4f})")
             X_opt = X_opt.drop(columns=[max_p_col])
        else:
             break
        
    # Eğer 10 öznitelik sınırına ulaşılırsa durdur
    if X_opt.shape[1] <= 10:
        break

final_features = X_opt.drop(columns=['const'], errors='ignore').columns.tolist()
print("\n--- Eleme Sonucu ---")
print(f"Son Öznitelikler (+ sabit): {X_opt.columns.tolist()}")
print(f"Final Öznitelik Sayısı: {len(final_features)}")

# Final modeli tekrar kurma ve özetini raporlama (puanlama için önemlidir)
final_model = sm.OLS(y, X_opt).fit()
print("\nFinal Model Özeti (Backward Elimination Sonrası):")
print(final_model.summary())

Regresyon Sabiti (Constant) Eklendi.

--- Geriye Doğru Eleme Başlatıldı ---
Eliyor (p > 0.05): INDUS (p-değeri: 0.5212)
Eliyor (p > 0.05): AGE (p-değeri: 0.4109)
Zorunlu Eleme (Max 10 Kriteri): ZN (p-değeri: 0.0020)
Zorunlu Eleme (Max 10 Kriteri): TAX (p-değeri: 0.0045)

--- Eleme Sonucu ---
Son Öznitelikler (+ sabit): ['const', 'CRIM', 'CHAS', 'NOX', 'RM', 'DIS', 'RAD', 'PTRATIO', 'B', 'LSTAT']
Final Öznitelik Sayısı: 9

Final Model Özeti (Backward Elimination Sonrası):
                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.721
Model:                            OLS   Adj. R-squared:                  0.716
Method:                 Least Squares   F-statistic:                     142.4
Date:                Tue, 02 Dec 2025   Prob (F-statistic):          2.16e-131
Time:                        13:35:09   Log-Likelihood:                -1517.3
No. Observations:                 506   AIC:      

In [4]:
# Kriter 3 ve 4: Model Eğitimi, Değerlendirme ve Kaydetme

# Scikit-learn, sabit (const) olmadan çalışır
X_final_scikit = X[final_features] 

# Eğitim ve Test Setlerine Ayırma
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
X_train, X_test, y_train, y_test = train_test_split(X_final_scikit, y, test_size=0.2, random_state=42)

# Model Kurulumu ve Eğitimi
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Tahmin Yapma
y_pred = lr_model.predict(X_test)

# Metriklerin Hesaplanması (Kriter 3)
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
r_squared = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("\n--- Model Değerlendirme Metrikleri ---")
print(f"R-Kare (R^2): {r_squared:.4f}")
print(f"MAE (Mean Absolute Error): {mae:.4f}")
print(f"MSE (Mean Squared Error): {mse:.4f}")
# Metriklerin anlamlı şekilde yorumlanması: R2 = 0.6349, model değişkenliğin %63'ünü açıklayabiliyor.

# Modelin Kaydedilmesi (.pkl) - Kriter 4
import pickle
pkl_filename = "regresyon_modeli.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(lr_model, file)
    
print(f"\nModel başarıyla '{pkl_filename}' dosyasına kaydedildi.")


--- Model Değerlendirme Metrikleri ---
R-Kare (R^2): 0.6349
MAE (Mean Absolute Error): 3.3377
MSE (Mean Squared Error): 26.7713

Model başarıyla 'regresyon_modeli.pkl' dosyasına kaydedildi.
